<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Multiple-Linear-Regression" data-toc-modified-id="Multiple-Linear-Regression-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Multiple Linear Regression</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Regression-with-Multiple-Predictors" data-toc-modified-id="Regression-with-Multiple-Predictors-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Regression with Multiple Predictors</a></span></li><li><span><a href="#Confounding-Variables" data-toc-modified-id="Confounding-Variables-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Confounding Variables</a></span></li><li><span><a href="#Dealing-with-Categorical-Variables" data-toc-modified-id="Dealing-with-Categorical-Variables-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Dealing with Categorical Variables</a></span><ul class="toc-item"><li><span><a href="#Dummying" data-toc-modified-id="Dummying-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Dummying</a></span></li></ul></li><li><span><a href="#Multiple-Regression-in-statsmodels" data-toc-modified-id="Multiple-Regression-in-statsmodels-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Multiple Regression in <code>statsmodels</code></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Diamonds-Dataset" data-toc-modified-id="Diamonds-Dataset-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Diamonds Dataset</a></span><ul class="toc-item"><li><span><a href="#Check-distribution-of-target" data-toc-modified-id="Check-distribution-of-target-6.0.1.1"><span class="toc-item-num">6.0.1.1&nbsp;&nbsp;</span>Check distribution of target</a></span></li><li><span><a href="#Build-model-with-log-scaled-target" data-toc-modified-id="Build-model-with-log-scaled-target-6.0.1.2"><span class="toc-item-num">6.0.1.2&nbsp;&nbsp;</span>Build model with log-scaled target</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Putting-it-in-Practice:-Wine-Dataset-🍷" data-toc-modified-id="Putting-it-in-Practice:-Wine-Dataset-🍷-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Putting it in Practice: Wine Dataset 🍷</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Running-the-Regression" data-toc-modified-id="Running-the-Regression-7.0.1"><span class="toc-item-num">7.0.1&nbsp;&nbsp;</span>Running the Regression</a></span></li></ul></li></ul></li><li><span><a href="#Scaling" data-toc-modified-id="Scaling-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Scaling</a></span></li><li><span><a href="#Multiple-Regression-in-Scikit-Learn" data-toc-modified-id="Multiple-Regression-in-Scikit-Learn-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Multiple Regression in Scikit-Learn</a></span><ul class="toc-item"><li><span><a href="#Sklearn-Metrics" data-toc-modified-id="Sklearn-Metrics-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Sklearn Metrics</a></span></li></ul></li><li><span><a href="#Level-Up:-Regression-with-Categorical-Features-with-the-Comma-Dataset" data-toc-modified-id="Level-Up:-Regression-with-Categorical-Features-with-the-Comma-Dataset-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Level Up: Regression with Categorical Features with the Comma Dataset</a></span></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from random import gauss
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats as stats

%matplotlib inline

![mlr](https://miro.medium.com/max/1280/1*lJKFo3yyZaFIx4ET1dLmlg.png)

# Multiple Linear Regression

# Objectives

- Conduct linear regressions in `statsmodels` and in `sklearn`
- Use the one-hot strategy to encode categorical variables

# Regression with Multiple Predictors

The main idea here is pretty simple. Whereas, in simple linear regression we took our dependent variable to be a function only of a single independent variable, here we'll be taking the dependent variable to be a function of multiple independent variables.

Our regression equation, then, instead of looking like $\hat{y} = mx + b$, will now look like:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x_1 + ... + \hat{\beta}_nx_n$.

Remember that the hats ( $\hat{}$ ) indicate parameters that are estimated.

Is this still a best-fit *line*? Well, no. What does the graph of, say, z = x + y look like? [Here's](https://academo.org/demos/3d-surface-plotter/) a 3d-plotter. (Of course, once we get x's with subscripts beyond 2 it's going to be very hard to visualize. But in practice linear regressions can make use of dozens or even of hundreds of independent variables!)

Is it possible to calculate the betas by hand? Yes, a multiple regression problem still has a closed-form solution:

In a word, for a multiple linear regression problem where $X$ is the matrix of independent variable values and $y$ is the vector of dependent variable values, the vector of optimizing regression coefficients $\vec{b}$ is given by:

$\vec{b} = (X^TX)^{-1}X^Ty$.

We'll focus more directly on matrix mathematics later in the course, so don't worry if this equation is opaque to you. See [here](https://stattrek.com/multiple-regression/regression-coefficients.aspx) for a nice explanation and example.

# Confounding Variables

Suppose I have a simple linear regression that models the growth of corn plants as a function of the temperature of the ambient air. And suppose there is a noticeable positive correlation between temperature and plant height.

In [ ]:
corn = pd.read_csv('data/corn.csv',
                  usecols=['temp', 'humid', 'height'])

In [ ]:
sns.lmplot(data=corn, x='temp', y='height')
plt.xlabel('Temperature ($\degree$ F)')
plt.ylabel('Height (cm)')
plt.title('Corn plant height as a function of temperature');

In [ ]:
corn.head()

It seems that higher temperatures lead to taller corn plants. But it's hard to know for sure. One **confounding variable** might be *humidity*. If we haven't controlled for humidity, then it's difficult to draw conclusions.

One solution is to use **both features** in a single model.

In [ ]:
sns.lmplot(data=corn, x='humid', y='height')
plt.xlabel('Humidity (%)')
plt.ylabel('Height (cm)')
plt.title('Corn plant height as a function of humidity');

In [ ]:
ax = plt.figure(figsize=(8, 6)).add_subplot(111, projection='3d')
ax.scatter(corn['temp'], corn['humid'], corn['height'],
           depthshade=True, s=40, color='#ff0000')
# create x,y
xx, yy = np.meshgrid(corn['temp'], corn['humid'])

# calculate corresponding z
z = 4.3825 * xx + 2.4693 * yy - 255.5434

# plot the surface
ax.plot_surface(xx, yy, z, alpha=0.01, color='#00ff00')

ax.view_init(30, azim=240)
ax.set_xlabel('Temperature ($\degree$ F)')
ax.set_ylabel('Humidity (%)')
ax.set_zlabel('Height (cm)')
plt.title('Corn plant height as a function of temperature and humidity');

One risk we run when adding more predictors to a model is that their correlations with the target may be nearly *collinear* with each other. This can make it difficult to determine which predictor is doing the heavy lifting. We shall explore this theme of **multicollinearity** in more depth in due course.

# Dealing with Categorical Variables

One issue we'd like to resolve is what to do with categorical variables, i.e. variables that represent categories rather than continua. In a Pandas DataFrame, these columns may well have strings or objects for values, but they need not. A certain heart-disease dataset from Kaggle, for example, has a target variable that takes values 0-4, each representing a different stage of heart disease.

## Dummying

One very effective way of dealing with categorical variables is to dummy them out. What this involves is making a new column for _each categorical value in the column we're dummying out_.

These new columns will be filled only with 0's and 1's, a 1 representing the presence of the relevant categorical value.

Let's look at a simple example:

In [ ]:
comma_use = pd.read_csv('data/comma-survey.csv')

For more on this dataset see [here](https://fivethirtyeight.com/features/elitist-superfluous-or-popular-we-polled-americans-on-the-oxford-comma/).

In [ ]:
comma_use.head()

In [ ]:
comma_use['In your opinion, which sentence is more gramatically correct?'].value_counts()

In [ ]:
comma_use.shape

In [ ]:
comma_use.isna().sum().sum()

In [ ]:
comma_use.dropna(inplace=True)

In [ ]:
comma_use.shape

In [ ]:
# Let's try using sklearn's OneHotEncoder to create our dummy columns:

ohe = OneHotEncoder(drop='first')
comma_trans = ohe.fit_transform(comma_use.drop('RespondentID', axis=1))

Could we have used ```pd.get_dummies()``` instead?

Well, yes. And in fact ```get_dummies()``` is in some ways easier; for one thing, it's built right into Pandas. But there are drawbacks with it as well. The main advantage of the `sklearn` tool is that it stores information about the columns and creates a persistent function that can be used on future data of the same form. See [this page](https://stackoverflow.com/questions/36631163/pandas-get-dummies-vs-sklearns-onehotencoder-what-are-the-pros-and-cons) for more.

In [ ]:
pd.get_dummies(comma_use.drop('RespondentID', axis=1))

So what did the encoder do?

In [ ]:
comma_trans

In [ ]:
comma_trans.todense()

In [ ]:
ohe.get_feature_names()

In [ ]:
comma_df = pd.DataFrame(comma_trans.todense(), columns=ohe.get_feature_names())
comma_df.head()

# Multiple Regression in `statsmodels`

Let's build a multiple regression with `statsmodels`. Let's start with a toy model:

In [ ]:
centers = np.arange(1, 6)
preds = np.array([stats.norm(loc=center, scale=3).rvs(200) for center in centers]).T
preds_df = pd.DataFrame(preds, columns=[f'var{center}' for center in centers])

target = preds_df['var1'] + 2*preds_df['var2'] + 3*preds_df['var3']\
    + 4*preds_df['var4'] + 5*preds_df['var5']
target_df = pd.DataFrame(target, columns=['target'])

In [ ]:
df = pd.concat([preds_df, target_df], axis=1)

df.head()

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

In [ ]:
model = sm.OLS(endog=y, exog=X).fit()

In [ ]:
model.summary()

### Diamonds Dataset

In [ ]:
data = sns.load_dataset('diamonds').drop(['cut', 'color', 'clarity'], axis = 1)

In [ ]:
data.head()

In [ ]:
X, y = data.drop('price', axis=1), data['price']

In [ ]:
model2 = sm.OLS(y, X).fit()
model2.summary()

In [ ]:
sm.graphics.plot_regress_exog(model2, 'carat', fig=plt.figure(figsize=(12, 8)));

#### Check distribution of target

In [ ]:
y.hist();

In [ ]:
y_scld = np.log(y)
y_scld.hist();

#### Build model with log-scaled target

In [ ]:
model3 = sm.OLS(y_scld, X).fit()
model3.summary()

In [ ]:
sm.graphics.plot_regress_exog(model3, 'carat', fig=plt.figure(figsize=(12, 8)));

**Remember that $R^2$ can be negative!**

In [ ]:
bad_pred = np.mean(y) * np.ones(len(y))
worse_pred = (np.mean(y) + 1000) * np.ones(len(y))

print(metrics.r2_score(y, bad_pred))
print(metrics.r2_score(y, worse_pred))

# Putting it in Practice: Wine Dataset 🍷

This dataset includes measurable attributes of different wines as well as their rated quality.

Imagine we want to attempt to estimate the perceived quality of a wine using these attributes.

In [ ]:
wine = pd.read_csv('data/wine.csv')

wine.head()

In [ ]:
wine['quality'].value_counts()

In [ ]:
wine['red_wine'].value_counts()

In [ ]:
wine.info()

In [ ]:
wine.describe()

### Running the Regression

First, we'll separate the data into our predictors (X) and target (y)

In [ ]:
wine_preds = wine.drop('quality', axis=1)
wine_target = wine['quality']
wine_preds.head()

Now we can perform our (multiple) linear regression! Since we already used `statsmodels`, let's use that again to fit the model and then check the summary:

In [ ]:
# use sm.add_constant() to add constant term/y-intercept
predictors = sm.add_constant(wine_preds)
predictors

In [ ]:
model = sm.OLS(wine_target, predictors).fit()
model.summary()

# Scaling

Before we construct a linear regression, let's *scale* our columns as z-scores. Why?

In a word, it's useful to have all of our variables be on the same scale, so that the resulting coefficients are easier to interpret. If the scales of the variables are very different one from another, then some of the coefficients may end up on very large or very tiny scales.

For more on this, see [this post](https://stats.stackexchange.com/questions/32649/some-of-my-predictors-are-on-very-different-scales-do-i-need-to-transform-them).

Let's try a model with our wine dataset now.

In [ ]:
# We'll include all the columns for now.

wine_preds_scaled = (wine_preds - np.mean(wine_preds)) / np.std(wine_preds)

In [ ]:
wine_preds_scaled.describe()

In [ ]:
predictors = sm.add_constant(wine_preds_scaled)
model = sm.OLS(wine_target, predictors).fit()
model.summary()

# Multiple Regression in Scikit-Learn

In [ ]:
# Let's create a StandardScaler object to scale our data for us.
ss = StandardScaler()


# Now we'll apply it to our data by using the .fit() and .transform() methods.

ss.fit(wine_preds)

wine_preds_st_scaled = ss.transform(wine_preds)

In [ ]:
# Check that the scaling worked about the same as when we did it by hand

np.allclose(wine_preds_st_scaled, wine_preds_scaled)

In [ ]:
wine_preds_scaled.head()

In [ ]:
wine_preds_st_scaled[:5, :]

In [ ]:
# Now we can fit a LinearRegression object to our training data!

lr = LinearRegression()
lr.fit(wine_preds_st_scaled, wine_target)

In [ ]:
# We can use the .coef_ attribute to recover the results
# of the regression.

lr.coef_

In [ ]:
lr.intercept_

In [ ]:
lr.score(wine_preds_st_scaled, wine_target)

In [ ]:
lr.predict(wine_preds_st_scaled)

## Sklearn Metrics

The metrics module in sklearn has a number of metrics that we can use to measure the accuracy of our model, including the $R^2$ score, the mean absolute error and the mean squared error. Note that the default 'score' on our model object is the $R^2$ score. Let's go back to our wine dataset:

In [ ]:
metrics.r2_score(wine_target, lr.predict(wine_preds_st_scaled))

Let's make sure this metric is properly calibrated. If we put simply $\bar{y}$ as our prediction, then we should get an $R^2$ score of *0*. And if we predict, say, $\bar{y} + 1$, then we should get a *negative* $R^2$ score.

In [ ]:
avg_quality = np.mean(wine_target)
num = len(wine_target)

metrics.r2_score(wine_target, avg_quality * np.ones(num))

In [ ]:
metrics.r2_score(wine_target, (avg_quality + 1) * np.ones(num))

In [ ]:
metrics.mean_absolute_error(wine_target, lr.predict(wine_preds_st_scaled))

In [ ]:
metrics.mean_squared_error(wine_target, lr.predict(wine_preds_st_scaled))

# Level Up: Regression with Categorical Features with the Comma Dataset

In [ ]:
comma_df.columns

In [ ]:
# We'll try to predict the first column of df: the extent to which
# the person accepts the sentence
# without the Oxford comma as more grammatically correct.

comma_target = comma_df['x0_It\'s important for a person to be honest, kind, and loyal.']

comma_predictors = comma_df[['x8_30-44',
       'x8_45-60', 'x8_> 60', 'x9_$100,000 - $149,999',
       'x9_$150,000+', 'x9_$25,000 - $49,999', 'x9_$50,000 - $99,999']]

comma_lr = LinearRegression()

comma_lr.fit(comma_predictors, comma_target)

In [ ]:
comma_lr.score(comma_predictors, comma_target)

In [ ]:
comma_lr.coef_

In [ ]:
comma_df.corr()['x0_It\'s important for a person to be honest, kind, and loyal.']

For more on the interpretation of regression coefficients for categorical variables, see [Erin's repo](https://github.com/hoffm386/coefficients-of-dropped-categorical-variables).